In [97]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

In [124]:
dataset_path = os.path.join('./data/motif_batik')
img_size = 150
batch_size = 80

In [128]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    validation_split = 0.2)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size = (img_size, img_size),
#     batch_size = batch_size,
    shuffle = True,
    class_mode = 'categorical',
    subset = 'training')

val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size = (img_size, img_size),
#     batch_size = batch_size,
    shuffle = True,
    class_mode = 'categorical',
    subset = 'validation')

Found 787 images belonging to 20 classes.
Found 194 images belonging to 20 classes.


In [120]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
#     # The third convolution
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
# #     # The fourth convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The fifth convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(20, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_78 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_79 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_79 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 flatten_30 (Flatten)        (None, 41472)             0         
                                                                 
 dropout (Dropout)           (None, 41472)           

In [126]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', 
              optimizer = Adam(learning_rate = 0.00001), 
              metrics=['accuracy'])

In [127]:
# Train the model
history = model.fit(train_generator, 
                    epochs=100, 
                    steps_per_epoch=20, 
                    validation_data = val_generator, 
                    verbose = 1, 
                    validation_steps=10)

Epoch 1/100
20/20 [==============================] - 28s 1s/step - loss: 2.8643 - accuracy: 0.1233 - val_loss: 2.8807 - val_accuracy: 0.1237
